In [3]:
!pip install datasets scikit-learn matplotlib pandas numpy transformers torch textacy keyphrase-vectorizers tqdm keybert g2p-en git+https://github.com/LIAAD/yake pytextrank rake-nltk spacy

  Cloning https://github.com/LIAAD/yake to /tmp/pip-req-build-_u87xo0i
  Running command git clone --filter=blob:none --quiet https://github.com/LIAAD/yake /tmp/pip-req-build-_u87xo0i
  Resolved https://github.com/LIAAD/yake to commit 0fa58cceb465162b6bd0cab7ec967edeb907fbcc
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of spacy-curated-transformers to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of spacy-transformers to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of spacy-transformers to determine which version is compatible with other requirements. This could take a while.
I

In [4]:
import pandas as pd
from keyphrase_vectorizers import KeyphraseCountVectorizer
from tqdm.auto import tqdm
import textacy
from textacy.extract.keyterms import scake
from datasets import load_dataset
import numpy as np
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keybert import KeyBERT
import yake
import spacy
import pytextrank
import torch
import json

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


/usr/local/lib/python3.10/dist-packages


# using scake for keyword extraction

invovles usage of semantic connectivity to compute word score for each keyword and outputs the keywords with the highest scores


In [5]:
with open('/content/drive/MyDrive/Colab Notebooks/sampleText.txt', 'r') as file:
  data = file.readlines()

print(data)

data_df = pd.DataFrame({'text' : data})
print(data_df)

['Explain how quantum computing can revolutionize modern cryptography, focusing on its impact on security protocols and encryption methods.\n', 'Discuss the role of artificial intelligence in healthcare, particularly in diagnosing diseases and improving patient outcomes through personalized treatments.\n', 'How can blockchain technology enhance supply chain transparency and efficiency for global logistics networks?\n', 'What challenges does machine learning face when applied to large-scale real-world datasets, and how can these be addressed?\n', 'Describe the potential impact of autonomous vehicles on reducing traffic congestion and improving road safety.\n', 'How will the Internet of Things (IoT) shape the future of smart cities and urban development?\n', 'Can edge computing help overcome the latency challenges of cloud computing in real-time applications?\n', 'What role do 5G networks play in enabling the next generation of mobile applications and connectivity?\n', 'Explain how cyber

In [6]:
scake_keywords_dict = dict()
for line in data:
  scake_keywords_dict[line] = list()

In [7]:
scake_keywords = []
for text in tqdm(data):
    doc = textacy.make_spacy_doc(text, lang='en_core_web_sm')
    scake_keywords.append(scake(doc, normalize='lower', topn=10))

  0%|          | 0/61 [00:00<?, ?it/s]

In [8]:
data_df['scake'] = pd.DataFrame([', '.join([keyword[0] for keyword in keywords]) for keywords in scake_keywords])
data_df.head()

,text,scake
0,Explain how quantum computing can revolutioniz...,"quantum computing, modern cryptography, securi..."
1,Discuss the role of artificial intelligence in...,"artificial intelligence, role, healthcare, pat..."
2,How can blockchain technology enhance supply c...,"supply chain transparency, technology, global ..."
3,What challenges does machine learning face whe...,"challenges, machine, face, scale real, world d..."
4,Describe the potential impact of autonomous ve...,"potential impact, autonomous vehicles, traffic..."


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# keybert

In [10]:
model_names = ["all-mpnet-base-v2", "all-distilroberta-v1", "all-MiniLM-L12-v2", "all-MiniLM-L6-v2"]
keybert = KeyBERT(model_names[0])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
vectorizer = KeyphraseCountVectorizer(spacy_pipeline='en_core_web_sm', pos_pattern='<J.*>*<N.*>+', stop_words='english', lowercase=True)

In [12]:
keybert_keywords = []
batch_size = 32

# Ensure the text column is treated as strings
data_df['text'] = data_df['text'].astype(str)

# Iterate through the data in batches
for idx in tqdm(range(0, len(data_df), batch_size)):
    # Calculate the batch end, handling the case where the batch is smaller than the set batch size
    batch_end = min(idx + batch_size, len(data_df))

    print(f"Processing batch from {idx} to {batch_end}...")  # Debugging statement

    # Extract current batch
    current_batch = data_df['text'].iloc[idx:batch_end]

    # Check the batch content
    print(f"Current batch content:\n{current_batch}")  # Debugging statement

    # Perform keyword extraction on the current batch
    try:
        extracted_keywords = keybert.extract_keywords(
            current_batch.tolist(),  # Convert to list to avoid any Pandas-related issues
            vectorizer=vectorizer,
            stop_words='english',
            top_n=10
        )
        keybert_keywords.extend(extracted_keywords)

    except Exception as e:
        print(f"Error during keyword extraction at batch {idx} to {batch_end}: {e}")

# Final keyword output
print(f"Extracted Keywords: {keybert_keywords}")

  0%|          | 0/2 [00:00<?, ?it/s]

2024-10-20 17:48:48,109 - KeyphraseVectorizer - INFO - It looks like you do not have downloaded a list of stopwords yet. It is attempted to download the stopwords now.
INFO:KeyphraseVectorizer:It looks like you do not have downloaded a list of stopwords yet. It is attempted to download the stopwords now.


Processing batch from 0 to 32...
Current batch content:
0     Explain how quantum computing can revolutioniz...
1     Discuss the role of artificial intelligence in...
2     How can blockchain technology enhance supply c...
3     What challenges does machine learning face whe...
4     Describe the potential impact of autonomous ve...
5     How will the Internet of Things (IoT) shape th...
6     Can edge computing help overcome the latency c...
7     What role do 5G networks play in enabling the ...
8     Explain how cybersecurity threats evolve in an...
9     What ethical concerns arise with the developme...
10    Discuss the potential of CRISPR gene editing f...
11    Discuss the impact of climate change on global...
12    Explain the mechanism of action of CRISPR-Cas9...
13    Analyze the effects of microplastics on marine...
14    Describe the role of gut microbiota in human h...
15    How do vaccines stimulate the immune system to...
16    Explore the principles of quantum entangle

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Processing batch from 32 to 61...
Current batch content:
32    Describe the role of neurotransmitters in ment...
33    Explain the process of protein synthesis and i...
34    Discuss the impact of deforestation on carbon ...
35    Explore the mechanisms of action of monoclonal...
36    What are the implications of gene therapy for ...
37    Discuss the role of bioinformatics in analyzin...
38    Explain how isotopes are used in radiometric d...
39    Describe the factors influencing the spread of...
40    Analyze the impact of agricultural practices o...
41    Discuss the significance of biodiversity hotsp...
42    Explain the role of enzymes in biochemical rea...
43    How do hormones regulate physiological process...
44    Explore the challenges of integrating renewabl...
45    Discuss the effects of global warming on ocean...
46    Explain the principles of evolutionary biology...
47    Analyze the impact of climate change on agricu...
48    Discuss the mechanisms of action of antir

In [13]:
data_df['keybert'] = pd.DataFrame([', '.join([keyword[0] for keyword in keywords]) for keywords in keybert_keywords])
data_df.head()

,text,scake,keybert
0,Explain how quantum computing can revolutioniz...,"quantum computing, modern cryptography, securi...","quantum computing, modern cryptography, encryp..."
1,Discuss the role of artificial intelligence in...,"artificial intelligence, role, healthcare, pat...","artificial intelligence, patient outcomes, hea..."
2,How can blockchain technology enhance supply c...,"supply chain transparency, technology, global ...","supply chain transparency, global logistics ne..."
3,What challenges does machine learning face whe...,"challenges, machine, face, scale real, world d...","world datasets, challenges, machine, scale, fa..."
4,Describe the potential impact of autonomous ve...,"potential impact, autonomous vehicles, traffic...","autonomous vehicles, road safety, traffic cong..."


# using YAKE

In [14]:
yake_extractor = yake.KeywordExtractor(
            lan = "en",
            n = 3,
            dedupLim = 0.9,
            dedupFunc = 'seqm',
            windowsSize = 1,
            top = 10,
            features = None
        )

In [15]:
yake_keywords = []
for idx, row in tqdm(data_df.iterrows(), total=len(data_df)):
    text = row['text']
    yake_keywords.append(yake_extractor.extract_keywords(text))

print(yake_keywords)

  0%|          | 0/61 [00:00<?, ?it/s]

[[('revolutionize modern cryptography', 0.0025258761156662004), ('Explain how quantum', 0.015380821171891606), ('modern cryptography', 0.015380821171891606), ('encryption methods', 0.015380821171891606), ('quantum computing', 0.02570861714399338), ('computing can revolutionize', 0.02570861714399338), ('revolutionize modern', 0.02570861714399338), ('impact on security', 0.02570861714399338), ('security protocols', 0.02570861714399338), ('protocols and encryption', 0.02570861714399338)], [('improving patient outcomes', 0.0042542192213185686), ('Discuss the role', 0.015380821171891606), ('intelligence in healthcare', 0.015380821171891606), ('personalized treatments', 0.015380821171891606), ('role of artificial', 0.02570861714399338), ('artificial intelligence', 0.02570861714399338), ('diagnosing diseases', 0.02570861714399338), ('diseases and improving', 0.02570861714399338), ('improving patient', 0.02570861714399338), ('patient outcomes', 0.02570861714399338)], [('global logistics networ

In [16]:
data_df['yake'] = pd.DataFrame([', '.join([keyword[0] for keyword in keywords]) for keywords in yake_keywords])
data_df.head()

,text,scake,keybert,yake
0,Explain how quantum computing can revolutioniz...,"quantum computing, modern cryptography, securi...","quantum computing, modern cryptography, encryp...","revolutionize modern cryptography, Explain how..."
1,Discuss the role of artificial intelligence in...,"artificial intelligence, role, healthcare, pat...","artificial intelligence, patient outcomes, hea...","improving patient outcomes, Discuss the role, ..."
2,How can blockchain technology enhance supply c...,"supply chain transparency, technology, global ...","supply chain transparency, global logistics ne...","global logistics networks, blockchain technolo..."
3,What challenges does machine learning face whe...,"challenges, machine, face, scale real, world d...","world datasets, challenges, machine, scale, fa...","large-scale real-world datasets, machine learn..."
4,Describe the potential impact of autonomous ve...,"potential impact, autonomous vehicles, traffic...","autonomous vehicles, road safety, traffic cong...","improving road safety, reducing traffic conges..."


#using TextRank

In [17]:
!pip install pytextrank

In [18]:
import spacy
import pytextrank

In [19]:
textrank_keywords = []

nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("textrank")

for idx, row in tqdm(data_df.iterrows(), total=len(data_df)):
  text = row['text']
  doc = nlp(text)

  phrase_list = []
  for phrase in doc._.phrases[:10]:
    phrase_list.append(phrase.text)

  textrank_keywords.append(phrase_list)

print(textrank_keywords)

  0%|          | 0/61 [00:00<?, ?it/s]

[['modern cryptography', 'security protocols', 'quantum computing', 'encryption methods', 'quantum', 'its impact'], ['artificial intelligence', 'patient outcomes', 'personalized treatments', 'healthcare', 'diseases', 'the role'], ['enhance supply chain transparency', 'global logistics networks', 'efficiency', 'technology'], ['face', 'large-scale real-world datasets', 'What challenges', 'these'], ['traffic congestion', 'autonomous vehicles', 'road safety', 'the potential impact'], ['smart cities', 'urban development', 'IoT', 'Things', 'the future', 'the Internet'], ['cloud computing', 'computing help', 'real-time applications', 'the latency challenges'], ['mobile applications', 'connectivity', 'the next generation', '5G networks', 'What role', '5'], ['cybersecurity threats', 'an increasingly connected world', 'what solutions', 'these risks'], ['facial recognition technology', 'ethical concerns', 'deployment', 'the development', 'What'], ['genetic disorders', 'CRISPR gene', 'CRISPR', 'th

In [20]:
data_df['textrank'] = pd.DataFrame([', '.join(keywords) for keywords in textrank_keywords])
data_df.head()

,text,scake,keybert,yake,textrank
0,Explain how quantum computing can revolutioniz...,"quantum computing, modern cryptography, securi...","quantum computing, modern cryptography, encryp...","revolutionize modern cryptography, Explain how...","modern cryptography, security protocols, quant..."
1,Discuss the role of artificial intelligence in...,"artificial intelligence, role, healthcare, pat...","artificial intelligence, patient outcomes, hea...","improving patient outcomes, Discuss the role, ...","artificial intelligence, patient outcomes, per..."
2,How can blockchain technology enhance supply c...,"supply chain transparency, technology, global ...","supply chain transparency, global logistics ne...","global logistics networks, blockchain technolo...","enhance supply chain transparency, global logi..."
3,What challenges does machine learning face whe...,"challenges, machine, face, scale real, world d...","world datasets, challenges, machine, scale, fa...","large-scale real-world datasets, machine learn...","face, large-scale real-world datasets, What ch..."
4,Describe the potential impact of autonomous ve...,"potential impact, autonomous vehicles, traffic...","autonomous vehicles, road safety, traffic cong...","improving road safety, reducing traffic conges...","traffic congestion, autonomous vehicles, road ..."


#using RAKE

In [21]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from rake_nltk import Rake

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [22]:
r = Rake()

In [23]:
rake_keywords = []

for idx, row in tqdm(data_df.iterrows(), total=len(data_df)):
  text = row['text']

  r.extract_keywords_from_text(text)
  keywords = r.get_ranked_phrases_with_scores()
  print(keywords)
  rake_keywords.append([', '.join(keyword[1] for keyword in keywords)])



print(rake_keywords)

  0%|          | 0/61 [00:00<?, ?it/s]

[(9.0, 'revolutionize modern cryptography'), (4.0, 'security protocols'), (4.0, 'quantum computing'), (4.0, 'encryption methods'), (1.0, 'impact'), (1.0, 'focusing'), (1.0, 'explain')]
[(9.0, 'improving patient outcomes'), (4.0, 'personalized treatments'), (4.0, 'diagnosing diseases'), (4.0, 'artificial intelligence'), (1.0, 'role'), (1.0, 'particularly'), (1.0, 'healthcare'), (1.0, 'discuss')]
[(36.0, 'blockchain technology enhance supply chain transparency'), (9.0, 'global logistics networks'), (1.0, 'efficiency')]
[(9.0, 'machine learning face'), (4.0, 'world datasets'), (4.0, 'scale real'), (1.0, 'large'), (1.0, 'challenges'), (1.0, 'applied'), (1.0, 'addressed')]
[(9.0, 'reducing traffic congestion'), (9.0, 'improving road safety'), (4.0, 'potential impact'), (4.0, 'autonomous vehicles'), (1.0, 'describe')]
[(4.0, 'urban development'), (4.0, 'smart cities'), (1.0, 'things'), (1.0, 'shape'), (1.0, 'iot'), (1.0, 'internet'), (1.0, 'future')]
[(15.0, 'edge computing help overcome'), 

In [24]:
data_df['rake'] = pd.DataFrame([', '.join(keyword for keyword in keywords) for keywords in rake_keywords])
data_df.head()

,text,scake,keybert,yake,textrank,rake
0,Explain how quantum computing can revolutioniz...,"quantum computing, modern cryptography, securi...","quantum computing, modern cryptography, encryp...","revolutionize modern cryptography, Explain how...","modern cryptography, security protocols, quant...","revolutionize modern cryptography, security pr..."
1,Discuss the role of artificial intelligence in...,"artificial intelligence, role, healthcare, pat...","artificial intelligence, patient outcomes, hea...","improving patient outcomes, Discuss the role, ...","artificial intelligence, patient outcomes, per...","improving patient outcomes, personalized treat..."
2,How can blockchain technology enhance supply c...,"supply chain transparency, technology, global ...","supply chain transparency, global logistics ne...","global logistics networks, blockchain technolo...","enhance supply chain transparency, global logi...",blockchain technology enhance supply chain tra...
3,What challenges does machine learning face whe...,"challenges, machine, face, scale real, world d...","world datasets, challenges, machine, scale, fa...","large-scale real-world datasets, machine learn...","face, large-scale real-world datasets, What ch...","machine learning face, world datasets, scale r..."
4,Describe the potential impact of autonomous ve...,"potential impact, autonomous vehicles, traffic...","autonomous vehicles, road safety, traffic cong...","improving road safety, reducing traffic conges...","traffic congestion, autonomous vehicles, road ...","reducing traffic congestion, improving road sa..."


# using spaCy


In [25]:
import spacy
from collections import Counter
from string import punctuation

In [26]:
nlp = spacy.load("en_core_web_sm")

In [27]:
def get_hotwords(text):
    result = []
    pos_tag = ['PROPN', 'ADJ', 'NOUN']
    doc = nlp(text.lower())
    for token in doc:
        if(token.text in nlp.Defaults.stop_words or token.text in punctuation):
            continue
        if(token.pos_ in pos_tag):
            result.append(token.text)
    return result

In [28]:
spacy_keywords = []

for idx, row in tqdm(data_df.iterrows(), total = len(data_df)):
  text = row['text']

  output = set(get_hotwords(text))
  most_common_list = Counter(output).most_common(5)
  spacy_keywords.append([', '.join(keyword[0] for keyword in most_common_list)])


print(spacy_keywords)

  0%|          | 0/61 [00:00<?, ?it/s]

[['encryption, methods, modern, security, cryptography'], ['treatments, outcomes, healthcare, artificial, intelligence'], ['technology, supply, chain, logistics, networks'], ['face, real, machine, large, scale'], ['vehicles, autonomous, impact, potential, traffic'], ['internet, future, cities, development, smart'], ['real, help, applications, challenges, cloud'], ['mobile, g, applications, generation, networks'], ['world, risks, solutions, cybersecurity, threats'], ['technology, concerns, facial, ethical, development'], ['crispr, gene, genetic, ethical, disorders'], ['ecosystem, stability, climate, biodiversity, impact'], ['crispr, applications, gene, potential, editing'], ['chains, microplastics, life, food, marine'], ['gut, microbiota, disease, health, human'], ['system, vaccines, immune, diseases, infectious'], ['entanglement, principles, implications, quantum, computing'], ['universe, matter, significance, dark, energy'], ['algorithms, outbreaks, disease, machine'], ['medicine, reg

In [29]:
data_df['spacy'] = pd.DataFrame([keyword for keyword in spacy_keywords])
data_df.head()

,text,scake,keybert,yake,textrank,rake,spacy
0,Explain how quantum computing can revolutioniz...,"quantum computing, modern cryptography, securi...","quantum computing, modern cryptography, encryp...","revolutionize modern cryptography, Explain how...","modern cryptography, security protocols, quant...","revolutionize modern cryptography, security pr...","encryption, methods, modern, security, cryptog..."
1,Discuss the role of artificial intelligence in...,"artificial intelligence, role, healthcare, pat...","artificial intelligence, patient outcomes, hea...","improving patient outcomes, Discuss the role, ...","artificial intelligence, patient outcomes, per...","improving patient outcomes, personalized treat...","treatments, outcomes, healthcare, artificial, ..."
2,How can blockchain technology enhance supply c...,"supply chain transparency, technology, global ...","supply chain transparency, global logistics ne...","global logistics networks, blockchain technolo...","enhance supply chain transparency, global logi...",blockchain technology enhance supply chain tra...,"technology, supply, chain, logistics, networks"
3,What challenges does machine learning face whe...,"challenges, machine, face, scale real, world d...","world datasets, challenges, machine, scale, fa...","large-scale real-world datasets, machine learn...","face, large-scale real-world datasets, What ch...","machine learning face, world datasets, scale r...","face, real, machine, large, scale"
4,Describe the potential impact of autonomous ve...,"potential impact, autonomous vehicles, traffic...","autonomous vehicles, road safety, traffic cong...","improving road safety, reducing traffic conges...","traffic congestion, autonomous vehicles, road ...","reducing traffic congestion, improving road sa...","vehicles, autonomous, impact, potential, traffic"


# keyword extraction using gemini

In [30]:
import google.generativeai as genai
import time

In [31]:
from google.colab import userdata
GEMINI_KEY = userdata.get('GEMINI_API_KEY')

In [32]:
genai.configure(api_key=GEMINI_KEY)

In [33]:
model = genai.GenerativeModel(model_name="gemini-1.0-pro")
convo = model.start_chat(history=[])

In [34]:
prompt_template = """
      You are keyword extractor. You will be provided with text and output at most
      10 keywords from that text ranked by quality and separated by comma,
      nothing else.
      """

In [35]:
def fetch_relevant_keywords(keywords, retries=3):
    for attempt in range(retries):
        try:
            prompt_template = """
            You are keyword extractor. You will be provided with text and output at most
            10 keywords from that text ranked by quality and separated by comma,
            nothing else.
            """
            model = genai.GenerativeModel(model_name="gemini-1.0-pro")
            convo = model.start_chat(history=[])
            response = convo.send_message(prompt_template + str(keywords))

            # Check if response is valid
            if response:
                return convo.last.text
            else:
                raise ValueError("Empty response received")

        except Exception as e:
            print(f"Error fetching relevant keywords on attempt {attempt + 1}: {e}")
            time.sleep(5)  # Backoff delay before retrying
    return ""

In [36]:
gemini_keywords = []

for idx, row in tqdm(data_df.iterrows(), total=len(data_df)):
  text = row['text']

  keywords = fetch_relevant_keywords(text)
  print(keywords)
  lst = keywords.split(", ")
  print(lst)

  gemini_keywords.append(lst)

  0%|          | 0/61 [00:00<?, ?it/s]

Quantum computing, cryptography, encryption, security protocols
['Quantum computing', 'cryptography', 'encryption', 'security protocols']
Artificial intelligence, Healthcare, Disease diagnosis, Personalized treatments, Patient outcomes
['Artificial intelligence', 'Healthcare', 'Disease diagnosis', 'Personalized treatments', 'Patient outcomes']
Blockchain technology, Supply chain transparency, Efficiency, Global logistics networks
['Blockchain technology', 'Supply chain transparency', 'Efficiency', 'Global logistics networks']
machine learning, large-scale datasets, challenges, real-world data, data volume, data variety, data complexity, data quality, data imbalance, interpretability
['machine learning', 'large-scale datasets', 'challenges', 'real-world data', 'data volume', 'data variety', 'data complexity', 'data quality', 'data imbalance', 'interpretability']
autonomous vehicles, traffic congestion, road safety, reduced crashes, improved traffic flow, enhanced efficiency, increased s

In [39]:
data_df['gemini'] = pd.DataFrame([', '.join(keyword for keyword in keywords_list) for keywords_list in gemini_keywords])
data_df.head()

,text,scake,keybert,yake,textrank,rake,spacy,gemini
0,Explain how quantum computing can revolutioniz...,"quantum computing, modern cryptography, securi...","quantum computing, modern cryptography, encryp...","revolutionize modern cryptography, Explain how...","modern cryptography, security protocols, quant...","revolutionize modern cryptography, security pr...","encryption, methods, modern, security, cryptog...","Quantum computing, cryptography, encryption, s..."
1,Discuss the role of artificial intelligence in...,"artificial intelligence, role, healthcare, pat...","artificial intelligence, patient outcomes, hea...","improving patient outcomes, Discuss the role, ...","artificial intelligence, patient outcomes, per...","improving patient outcomes, personalized treat...","treatments, outcomes, healthcare, artificial, ...","Artificial intelligence, Healthcare, Disease d..."
2,How can blockchain technology enhance supply c...,"supply chain transparency, technology, global ...","supply chain transparency, global logistics ne...","global logistics networks, blockchain technolo...","enhance supply chain transparency, global logi...",blockchain technology enhance supply chain tra...,"technology, supply, chain, logistics, networks","Blockchain technology, Supply chain transparen..."
3,What challenges does machine learning face whe...,"challenges, machine, face, scale real, world d...","world datasets, challenges, machine, scale, fa...","large-scale real-world datasets, machine learn...","face, large-scale real-world datasets, What ch...","machine learning face, world datasets, scale r...","face, real, machine, large, scale","machine learning, large-scale datasets, challe..."
4,Describe the potential impact of autonomous ve...,"potential impact, autonomous vehicles, traffic...","autonomous vehicles, road safety, traffic cong...","improving road safety, reducing traffic conges...","traffic congestion, autonomous vehicles, road ...","reducing traffic congestion, improving road sa...","vehicles, autonomous, impact, potential, traffic","autonomous vehicles, traffic congestion, road ..."


# evaluation of all methods

In [40]:
results = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sampleTextResults.csv')
print(results)

                                                 text  \
0   Explain how quantum computing can revolutioniz...   
1   Discuss the role of artificial intelligence in...   
2   How can blockchain technology enhance supply c...   
3   What challenges does machine learning face whe...   
4   Describe the potential impact of autonomous ve...   
..                                                ...   
56  Analyze the role of public health policies in ...   
57  Discuss the importance of ethical consideratio...   
58  Explain the mechanisms of action of vaccines i...   
59  Analyze the impact of climate change on freshw...   
60  Discuss the relationship between genetics and ...   

                                                words  
0   quantum computing, modern cryptography, securi...  
1   artificial intelligence, healthcare, diagnosis...  
2   blockchain, technology, chain transparency, ef...  
3   machine learning, challenges, large-scale, rea...  
4   impact, autonomous vehicles, re

In [41]:
true_keywords = list()
for index, line in results.iterrows():
    true_keywords.append(line['words'])

print(true_keywords)

data_df['true_keywords'] = true_keywords
print(data_df.head())

['quantum computing, modern cryptography, security protocols, encryption methods', 'artificial intelligence, healthcare, diagnosis, diseases, improving, personalized treatments', 'blockchain, technology, chain transparency, efficiency, global logistics networks', 'machine learning, challenges, large-scale, real-world datasets', 'impact, autonomous vehicles, reducing traffic congestion, improving road safety', 'internet of things, smart cities, urban development, future', 'edge computing, latency challgnes, cloud computing, real-time apps', '5G networks, next generation, mobile apps, connectivity', 'cybersecurity threats, risks', 'ethical concerns, development, deployment, facial recognition, technology', 'CRISPR gene editing, treating genetic disorders, ethical implications', 'climate change, global biodiversity, ecosystem stability', 'mechanism, CRISPR-Cas9, gene eciting, applications', 'microplastics, marine life, food chains', 'gut microbiota, human health and disease', 'vaccines, s

In [43]:
def calculate_precision(extracted_keywords):
  precision_scores = list()
  recall_scores = list()
  f1_scores = list()
  for index, row in data_df.iterrows():
    true_keywords = row['true_keywords']
    extracted_keywords = row[extracted_keywords]

    extracted_set = set(extracted_keywords)
    true_set = set(true_keywords)

    true_positives = len(extracted_set & true_set)
    false_positives = len(extracted_set - true_set)
    false_negatives = len(true_set - extracted_set)

    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)

  return precision_scores, recall_scores, f1_scores

In [50]:
import pandas as pd

# Define a list of the keyword extraction method column names
keyword_columns = ['scake', 'yake', 'keybert', 'textrank', 'spacy', 'gemini']

# Initialize lists to store the results for each method
avg_precision_list = []
avg_recall_list = []
avg_f1_list = []

# Function to calculate average scores for precision, recall, and F1
def calculate_average(scores):
    return sum(scores) / len(scores) if len(scores) > 0 else 0

# Loop through each keyword extraction method and calculate the scores
for method in keyword_columns:
    precision, recall, f1 = calculate_precision(f'{method}')

    # Calculate the average scores for this method
    avg_precision = calculate_average(precision)
    avg_recall = calculate_average(recall)
    avg_f1 = calculate_average(f1)

    # Append the results to the corresponding lists
    avg_precision_list.append(avg_precision)
    avg_recall_list.append(avg_recall)
    avg_f1_list.append(avg_f1)

# Create a DataFrame to store the average results
evaluation = pd.DataFrame({
    'Method': keyword_columns,
    'Avg Precision': avg_precision_list,
    'Avg Recall': avg_recall_list,
    'Avg F1': avg_f1_list
})

     Method  Avg Precision  Avg Recall    Avg F1
0     scake       0.813115    0.218269  0.343586
1      yake       0.672131    0.144554  0.237574
2   keybert       0.669399    0.215607  0.325549
3  textrank       0.723653    0.271886  0.394504
4     spacy       0.842623    0.225254  0.354924
5    gemini       0.904918    0.241331  0.380493


In [52]:
evaluation

,Method,Avg Precision,Avg Recall,Avg F1
0,scake,0.813115,0.218269,0.343586
1,yake,0.672131,0.144554,0.237574
2,keybert,0.669399,0.215607,0.325549
3,textrank,0.723653,0.271886,0.394504
4,spacy,0.842623,0.225254,0.354924
5,gemini,0.904918,0.241331,0.380493


### gemini shows the best accuracy, and spacy next